# Recommender Application
This notebook contains the implementation of the recommender application. It imports the recommender algorithm and assumes the model has already been trained with a dataset. 

The libraries required to run this application are:
- tkinter
- pandas
- recommender3 (custom library)

In [3]:
#import recommender algorithm and UI libraries
import tkinter as tk
from tkinter import ttk
import pandas as pd
from recommender3 import create_user_review_matrix, recommend_movies, update_reviews

We need to load the below datasets to be used when initializing the application and as required for recommender3 library methods:
- clean_movies_id_appended.csv
- movies_preprocessed.csv'
- user_reviews_200moviesplus.csv
- clean_movies_id_appended.csv

In [5]:
#load data
data = pd.read_csv('clean_movies_id_appended.csv', header = 0)
movie_features = pd.read_csv('movies_preprocessed.csv', index_col='movieId')
reviews = pd.read_csv('user_reviews_200moviesplus.csv')
movie_titles = pd.read_csv('clean_movies_id_appended.csv', usecols=['movieId', 'original_title'], index_col='movieId')

These properties will be used to intialize and run the MovieRecommender application. **Update the newUserId** each time you run the app. 

In [7]:
#properties to be used in MovieRecommender class initialization
titles = data['original_title']
movieIdList = data['movieId']
newUserId = 1362

In the next 3 sections, we are defining the 3 main python classes needed to build and run the application
- Rating: used to store an individual user rating
- UserProfile: represents a user and the list of ratings associated with them
- MovieRecommender: the core application class with functionality to submit a rating and display recommendations

In [9]:
#Rating Class
class Rating:
    
    def __init__(self, movie, score):
        self.userId = newUserId
        self.movie = movie
        self.score = score

In [10]:
#UserProfile class
class UserProfile:
    
    def __init__(self):
        self.ratings = []

    def addRating(self, movieRating):
        self.ratings.append(movieRating)
        update_reviews(dict([(movieRating.movie, (movieRating.userId, movieRating.score))]))
                
    def displayRatings(self):
        for rating in self.ratings:
            print(f"{rating.movie} {rating.score}")

In [11]:
# Recommender Class
class MovieRecommender:
    
    def __init__(self, root, movieNames, movieIds):
        #initialize base variables
        self.root = root
        self.root.title('Movie Recommendation App')
        self.root.geometry("1200x800")
        self.user = UserProfile()
        self.sampleData = movieNames
        self.movieIds = movieIds

        #add greeting message at the top of the app
        self.helloLabel = tk.Label(root, text="Hello user! Welcome to the movie recommender app", 
                                   font=("Arial", 14))
        self.helloLabel.pack(pady=10)

        #add functionality to display movie title and ask for a rating
        self.totalRated = 0
        self.ratingsRemainingLabel = tk.Label(root, text=f"Ratings Remaining: {10 - self.totalRated}")
        self.ratingsRemainingLabel.pack(pady=10)
        
        self.currentMovie = 0
        self.movieLabel = tk.Label(root, 
                                   text=f"Rate the following movie: \n {self.sampleData[self.currentMovie]}")
        self.movieLabel.pack(pady=10)

        #add a rating scale of 1 - 5
        self.ratingScaleFrame = tk.Frame(root)
        self.ratingScaleFrame.pack(pady=10, fill="x", padx=40)
        
        self.minRatingLabel = tk.Label(self.ratingScaleFrame, text="1\nPoor")
        self.minRatingLabel.pack(side="left")
        
        self.movieRating = tk.DoubleVar(value=3)
        self.currentRatingLabel = tk.Label(root, 
                                   text=f"Current Rating: \n {self.movieRating}")
        self.currentRatingLabel.pack(pady=10)
        
        self.ratingScale = ttk.Scale(
            self.ratingScaleFrame,
            from_=1,
            to=5,
            orient="horizontal",
            variable=self.movieRating,
            length=20,
            command=self.onRatingChange
        )
        self.ratingScale.set(3)
        self.ratingScale.pack(side="left", padx=10, fill="x", expand=True)
        
        self.maxRatingLabel = tk.Label(self.ratingScaleFrame, text="5\nExcellent")
        self.maxRatingLabel.pack(side="right")

        #skip movie button
        self.skipButton = tk.Button(root, text="Skip", command=self.onSkipButtonClick)
        self.skipButton.pack(pady=10)

        #submit rating button
        self.submitButton = tk.Button(root, text="Submit", command=self.onSubmitButtonClick)
        self.submitButton.pack(pady=10)

        #display movie recommendations
        self.recommendationLabel = tk.Label(root, text="Here are your recommendations:")
        self.movieRecList = tk.Listbox(root, height=20, width=50)

    #when the user clicks skip
    #show the next movie or display recommendations if app is at the end of the movie list
    def onSkipButtonClick(self):
        if self.currentMovie < len(self.sampleData) - 1:
            self.currentMovie+=1
            self.movieLabel.config(text=f"Rate the following movie: \n {self.sampleData[self.currentMovie]}")
        else: 
            self.hideRatingShowRecs()
            self.generateMovieRecs()

    #when the user clicks submit
    #show the next movie or display recommendations 
    #if app is at the end of the movie list or they have submitted 5 ratings
    def onSubmitButtonClick(self): 
        newRating = Rating(self.movieIds[self.currentMovie], self.movieRating)
        self.user.addRating(newRating)
        self.totalRated+=1
        if self.totalRated - 10 == 0:
            self.hideRatingShowRecs()
            self.generateMovieRecs()
        elif self.currentMovie <= len(self.sampleData) - 1:
            self.currentMovie+=1
            self.movieLabel.config(text=f"Rate the following movie: \n {self.sampleData[self.currentMovie]}")
            self.ratingsRemainingLabel.config(text=f"Ratings Remaining: {10 - self.totalRated}")
        else: 
            self.hideRatingShowRecs()
            self.generateMovieRecs()

    #ensure movie ratings are in increments of 0.5
    def onRatingChange(self, val):
        valAsFloat = float(val)
        roundedToPoint5 = round(valAsFloat * 2)/2
        self.movieRating = roundedToPoint5
        self.currentRatingLabel.config(text=f"Current Rating: \n {self.movieRating}")

    #display recommendations from model
    def showMovieRecs(self, recs):
        for rec in recs:
            movieIdIndex = list(self.movieIds).index(rec[0])
            self.movieRecList.insert(tk.END, self.sampleData[movieIdIndex])
        self.movieRecList.pack()

    #get recommendations from model
    def generateMovieRecs(self):
        print("Generating Recommendations")
        reviews = pd.read_csv('user_reviews_200moviesplus.csv')
        review_matrix = create_user_review_matrix(reviews, newUserId)
        recs = recommend_movies(newUserId, movie_features, movie_titles, review_matrix, 20)
        self.showMovieRecs(recs)

    #hide rating prompt when recommendations are displayed
    def hideRatingShowRecs(self):
        self.skipButton.pack_forget()
        self.movieLabel.pack_forget()
        self.ratingsRemainingLabel.pack_forget()
        self.ratingScaleFrame.pack_forget()
        self.submitButton.pack_forget()
        self.ratingScaleFrame.pack_forget()
        self.currentRatingLabel.pack_forget()
        self.recommendationLabel.pack(pady=10)
    

Finally we can run the application with the below code snippet and the output contains the associated movieId and predicted rating for the user.

Also note a UI quirk: When "Here are your recommendations" displays, you must click on the app again for the recommendation list to appear. 

In [13]:
#run application
if __name__ == '__main__':
    root = tk.Tk()
    app = MovieRecommender(root, titles, movieIdList)
    root.mainloop()

Generating Recommendations


/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Movie: 4993, Rating: 4.478389354678479
Movie: 7153, Rating: 4.433846422050134
Movie: 5952, Rating: 4.387764999207432
Movie: 60069, Rating: 4.268477790861767
Movie: 114662, Rating: 4.257618031632741
Movie: 58559, Rating: 4.207834152747284
Movie: 45722, Rating: 4.174130082944875
Movie: 8368, Rating: 4.123508648952141
Movie: 98809, Rating: 4.116970227888996
Movie: 54001, Rating: 4.099246636495145
Movie: 59315, Rating: 4.050564124347248
Movie: 106487, Rating: 4.042794748151508
Movie: 79132, Rating: 4.036188346914965
Movie: 40815, Rating: 4.032861343625565
Movie: 106489, Rating: 4.025086772712558
Movie: 6377, Rating: 4.006598821541164
Movie: 110102, Rating: 4.004856537884529
Movie: 135133, Rating: 4.00335353915706
Movie: 5816, Rating: 3.999527505412597
Movie: 134130, Rating: 3.963336403549016
